# Project
*Team #1 - Connie Dunlap, Ryne Krueger, Jackson Robbins, Wyatt Tauber*

*CSEC-520/620 Cyber Analytics & Machine Learning*

## Prerequisites

This notebook requires the following packages:
* TBD

The following files and folder should be in the same directory as this notebook:
* TBD

## 1 Dataset Processing

TBD

In [6]:
"""
1 Dataset Processing
Author: Wyatt Tauber

TBD
"""
import csv
import os
import subprocess

for dir in os.listdir():
    if '.' not in dir:
        #print(dir)
        for file in os.listdir(dir):
            print(dir + "/" + file)
        
        command = "rahash2 -a entropy"
        string = "/1dd620a7293d700b1f15d48089a558aede814f53627dd87af2e77ca4fc6f94f5"
        proc = subprocess.Popen([command + " " + string], stdout=subprocess.PIPE, shell=True)
    
        (out, err) = proc.communicate()
        
        #print("program output:", out)

        file_info = out.decode().strip().split()
        file_hash = file_info[0].split("/")[-1][0:-1]
        file_entropy = file_info[-1]
        print("hash:", file_hash)
        print("entropy:", file_entropy)
        

UPX/unpacked_0a197532014ccac87ea51774790fad1b4c5ca7a27e7aec4222054a2034e20e66
UPX/unpacked_4522e14192188c5d82d1f7444175c368ecae873afeb7c27afe1ffd0acd9f5344
UPX/unpacked_1c12e5d9250d460be9779e89a0cc388d732f62044ff9a12d83802c906a5b5151
UPX/2619f2b4a6a473ccd3064ed7828471dd6fd05269041fe07034fb5d6afca55590
UPX/unpacked_1c187c7565ccde403cb9ffab410fbd1c6256d4e224ba72c8356e0c8d1cdb40cf
UPX/1fb378babd2609b7c37dc1b5b3a78acddf749a4cad066ee036e3ff9348b13af8
UPX/0f29cbfc7996b859e44750bcc2c10a3918bf0a6b64d7d8cde20e4ca37f805d14
UPX/5faf817628dd40d3443aac4ae7a0f89ffc22ef9d92eef69326dbee8cbdd5b6b1
UPX/1a17d2459c68894aaf6e38bd94b67fe31649cda5673899ec23506ce1a55ea3b3
UPX/7597d65e0314787246b38459a629f021b3abc27af1e88361cc364998122163cf
UPX/1d54bbc67caa6d943c4824bf716f6e7f36e904bf009592203baf01c68cbe7529
UPX/0cf5e4fccba912ae18676319fff3c1908a2683286bcc44f8dcab5e6d9a7d02b5
UPX/unpacked_0ed3adb7576b8b076dae607550cc6cf7f435773e16c233f02b14c235a92c2234
UPX/unpacked_0aee6832e9417f1c04d219040a9e98594b20dd59b5f0d